In [1]:
from PySide2 import QtCore, QtGui, __version__
%gui qt
# %load_ext autoreload
# %autoreload 2

In [2]:

import napari
import numpy as np
import fdtd
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
# fdtd.set_backend("numpy")
import torch

fdtd.set_backend("torch.cuda")

from fdtd.backend import backend as bd

In [3]:
# the whole thing in function form:

# specs I grabbed from a random edmund optic asphere
LENS = {
    'radius': 5, # radius of the lens
    'curve': 1.982161E-01, # main curvature
    'k': -1.055033 ,
    'As': [
        0,             # A2
        5.096600E-04,  # A4
        2.254951E-06,  # A6
        8.064402E-09,  # A8
        -9.079062E-10] # A10
}

def asphere(n, radius=5, curve=2, k=-1, As=[]):
    """returns a lens surface with `n` samples
    """
    s = np.linspace(-radius, radius, n)
    z = curve * s**2 / (1 + np.sqrt(1 - (1 + k) * curve**2 * s**2))
    for n, A in enumerate(As):
        z += A*s**(n*2 + 2)
    return z

def onhost(arr):
    if hasattr(arr, 'cpu'):
        return np.array(arr.cpu())
    return arr

def setup_grid(
    width = 20e-6,
    height = 15e-6,
    depth = 15e-6,
    pixel = 100e-9,
    wavelength = 1000e-9,
    lens=LENS,
    lens_scale=1,
    lens_position=6.5,
    extra_glass=1,
    refractive_index = 1.68,
    bes=None
):
    if bes:
        # enforce odd pixels for symmetry
        height += pixel
    grid = fdtd.Grid(shape = (height, width, depth),grid_spacing = pixel)
    # to avoid reflections at the edges, add "perfectly matched layers" (PML)
    grid[0:10, :, :] = fdtd.PML(name="pml_xlow")
    grid[-10:, :, :] = fdtd.PML(name="pml_xhigh")
    grid[:, 0:10, :] = fdtd.PML(name="pml_ylow")
    grid[:, -10:, :] = fdtd.PML(name="pml_yhigh")
    grid[:, :, 0:10] = fdtd.PML(name="pml_zlow")
    grid[:, :, -10:] = fdtd.PML(name="pml_zhigh")

    if bes:
        grid[grid.Nx//2 - bes:grid.Nx//2 - bes+2, 13, 0] = fdtd.LineSource(period=wavelength/299792458, name="p1")
        grid[grid.Nx//2 + bes:grid.Nx//2 + bes+2, 13, 0] = fdtd.LineSource(period=wavelength/299792458, name="p2")
    else:
        for z in range(10,grid.Nz-10):
            grid[10:grid.Nx-10, 11, z] = fdtd.LineSource(period=wavelength/299792458, name=f"source{z}")
        
    surface = asphere(grid.Nx, **lens)
    _pix = (lens['radius']*2) / (grid.Nx-1) # mm
    surface /= _pix
    ny = int(surface.max() + (extra_glass/_pix))
    _lens, yy = np.mgrid[:len(surface), :ny]
    _lens = np.zeros_like(yy)
    _lens[yy>surface[:, np.newaxis]] = 1
    _perm = bd.ones((*_lens.shape,1))
    _perm += bd.array(_lens[:,:,None])*(refractive_index**2 - 1)
    lens_start = int(_lens.shape[0]/lens_position)
    lensobj = fdtd.Object(permittivity=_perm, name="lens")
    grid[:, lens_start:lens_start + _lens.shape[1], :] = lensobj
    return grid

def capture_frames(grid, total_time=1000, dt=10):
    frames = []
    grid.reset()
    for t in tqdm(np.arange(0, total_time, dt)):
        grid.run(total_time=dt, progress_bar=False)
        frames.append(onhost(bd.sum(grid.E, -1)))
    return np.stack(frames)

In [5]:
mygrid = setup_grid()
print(mygrid)
frms = capture_frames(mygrid, 400, 10)
print(frms.shape)

Grid(shape=(150,200,150), grid_spacing=1.00e-07, courant_number=0.57)

sources:
    LineSource(period=17, power=1.0, phase_shift=0.0, name='source10')
        @ x=[10, ... , 139], y=[11, ... , 11], z=[10, ... , 10]
    LineSource(period=17, power=1.0, phase_shift=0.0, name='source11')
        @ x=[10, ... , 139], y=[11, ... , 11], z=[11, ... , 11]
    LineSource(period=17, power=1.0, phase_shift=0.0, name='source12')
        @ x=[10, ... , 139], y=[11, ... , 11], z=[12, ... , 12]
    LineSource(period=17, power=1.0, phase_shift=0.0, name='source13')
        @ x=[10, ... , 139], y=[11, ... , 11], z=[13, ... , 13]
    LineSource(period=17, power=1.0, phase_shift=0.0, name='source14')
        @ x=[10, ... , 139], y=[11, ... , 11], z=[14, ... , 14]
    LineSource(period=17, power=1.0, phase_shift=0.0, name='source15')
        @ x=[10, ... , 139], y=[11, ... , 11], z=[15, ... , 15]
    LineSource(period=17, power=1.0, phase_shift=0.0, name='source16')
        @ x=[10, ... , 139], y=[11, ...


(40, 150, 200, 150)


In [8]:
napari.view_image(frms.transpose(1,0,3,2))

(100, 100, 100)

In [4]:
napari.view_image(a)

In [10]:
q = np.arange(3*4*5).reshape(3,4,5)
q.shape
print(q)
napari.view_image(q)

[[[ 0  1  2  3  4]
  [ 5  6  7  8  9]
  [10 11 12 13 14]
  [15 16 17 18 19]]

 [[20 21 22 23 24]
  [25 26 27 28 29]
  [30 31 32 33 34]
  [35 36 37 38 39]]

 [[40 41 42 43 44]
  [45 46 47 48 49]
  [50 51 52 53 54]
  [55 56 57 58 59]]]


In [ ]:
q[]